# <a id='toc1_'></a>[Парсинг вакансий аналитика с hh.ru в Москве](#toc0_)

**Содержание**<a id='toc0_'></a>    
- [Парсинг вакансий аналитика с hh.ru в Москве](#toc1_)    
  - [Подготовка к пасрингу](#toc1_1_)    
  - [Выгрузка вакансий](#toc1_2_)    
  - [Выгрузка доп. информации по выборке вакансий](#toc1_3_)    
  - [Информация по формату, опыту и зарплате](#toc1_4_)    
  - [Анализ ключевых навыков](#toc1_5_)    
  - [Анализ описания вакансий](#toc1_6_)    

<!-- vscode-jupyter-toc-config
	numbering=false
	anchor=true
	flat=false
	minLevel=1
	maxLevel=6
	/vscode-jupyter-toc-config -->
<!-- THIS CELL WILL BE REPLACED ON TOC UPDATE. DO NOT WRITE YOUR TEXT IN THIS CELL -->

## <a id='toc1_1_'></a>[Подготовка к пасрингу](#toc0_)

In [1]:
# импорт библиотек
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
import time
import re
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
url = 'https://hh.ru/vacancies/analitik'

In [3]:
cookies = {
    '__ddg1_': 'EUFoJ49oO7uKyGESsDAg',
    '_xsrf': '565662c50cd7ac37d193aee93439b12f',
    'hhrole': 'anonymous',
    'regions': '1',
    'region_clarified': 'NOT_SET',
    'display': 'desktop',
    'crypted_hhuid': 'F52DCC991562307604485239C6A45FED903F6A00FC346567CD4B5B0AA4AF30C8',
    'hhtoken': 'KjSbZcLBxaKz3fzi0kegnrcBCo7o',
    'hhuid': 'wcp6n_2aGk5lUWXXQoE7Uw--',
    'total_searches': '4',
    'GMT': '3',
    'device_magritte_breakpoint': 'm',
    'device_breakpoint': 'm',
    'iap.uid': 'c067c7dbb3fa40f2893d158238a2227b',
    '_ym_uid': '1708606083617511105',
    '_ym_d': '1708606083',
    'tmr_lvid': '93591b028af19745c4a181174a74c60f',
    'tmr_lvidTS': '1708606082869',
    'cfidsgib-w-hh': 'KnxKquiPOjhVJmx5/JBDDJnTdW+bRgpvjriZ60coV8quZ0DcnKgXQMRFfGP2HRCTVo6cEehAX3DAWTlgNbAlCrfM5N/TXg9w/+/51ORHHDpjoFRqlhK04qUjmyfnjSPjvlSIdAexQkdG9rE0zNr978II2Mu6zaUwA8H0hg==',
    '_ym_isad': '2',
    '__zzatgib-w-hh': 'MDA0dBA=Fz2+aQ==',
    'cfidsgib-w-hh': 'CCvJ1thhtibnwTm/J++4gUnA5v5Qtmi3r482ZxLxlyw9AP/xVjqClvMsJrjmTqtg9LBFWJcYKt//svfo4plL4UNoWoFTbULct5GedPmYfBJvHJ6PnY4nV7UgvjjhMs+4RG6eL3tqOmnxB3VmhL5A494yHx6YfbRt/iAHiw==',
    '_ym_visorc': 'b',
    'gsscgib-w-hh': 'zLG0xSgas3Xl7T5MyY8XPDycx1vIdJOEZaSMbSmzBMifW7bvwEPWDgROKVo+S35SxnKvDzrhq9alJhnFwb/1l0aGlQqkGja3f6yrNB1APX4nP7Kke6xFfFHjQ3XvZXoVSqKgHzOLR79VsmljVLYE4yago/USE7n/++waXH46zuqxqeX/y0iXAmh74N1twEb69DWPbGnq837vEbyMbUxXlhWVbmXIXJspZWqM2VCK2AtveQLEbVLptL22lzcNQQ==',
    'gsscgib-w-hh': '1VqwfwRSMN0XqLDy5M61UV/blVoA8iLyCrEwZmWq/wnCVDSH6gEZ8BNmD7EriredJaKHDDzdVkQUO2z9phSBY0lrd+iIeevw+h0nYdxRF2cMinY0J8tOjznKW0nvOkaf6D6NOzacWpikeLZV08sq2iXnULA15SKV4KxIZ95GJM4zaXP0aaoybeerYnio6jKYcVH55vhE7BPxu1SKscmiXPY/5gB0+/Q0xJs+7CdJy8e0jToIC1J3XOOWjZYlxw==',
    'fgsscgib-w-hh': 'RZjje55ccb9683638fe411189d9c9891c4317354',
    'tmr_detect': '0^%^7C1708606149267',
    'cookie_policy_agreement': 'true',
}

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:123.0) Gecko/20100101 Firefox/123.0',
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,*/*;q=0.8',
    'Accept-Language': 'ru-RU,ru;q=0.8,en-US;q=0.5,en;q=0.3',
    # 'Accept-Encoding': 'gzip, deflate, br',
    'Referer': 'https://hh.ru/vacancies/analitik',
    'Connection': 'keep-alive',
    # 'Cookie': '__ddg1_=EUFoJ49oO7uKyGESsDAg; _xsrf=565662c50cd7ac37d193aee93439b12f; hhrole=anonymous; regions=1; region_clarified=NOT_SET; display=desktop; crypted_hhuid=F52DCC991562307604485239C6A45FED903F6A00FC346567CD4B5B0AA4AF30C8; hhtoken=KjSbZcLBxaKz3fzi0kegnrcBCo7o; hhuid=wcp6n_2aGk5lUWXXQoE7Uw--; total_searches=4; GMT=3; device_magritte_breakpoint=m; device_breakpoint=m; iap.uid=c067c7dbb3fa40f2893d158238a2227b; _ym_uid=1708606083617511105; _ym_d=1708606083; tmr_lvid=93591b028af19745c4a181174a74c60f; tmr_lvidTS=1708606082869; cfidsgib-w-hh=KnxKquiPOjhVJmx5/JBDDJnTdW+bRgpvjriZ60coV8quZ0DcnKgXQMRFfGP2HRCTVo6cEehAX3DAWTlgNbAlCrfM5N/TXg9w/+/51ORHHDpjoFRqlhK04qUjmyfnjSPjvlSIdAexQkdG9rE0zNr978II2Mu6zaUwA8H0hg==; _ym_isad=2; __zzatgib-w-hh=MDA0dBA=Fz2+aQ==; cfidsgib-w-hh=CCvJ1thhtibnwTm/J++4gUnA5v5Qtmi3r482ZxLxlyw9AP/xVjqClvMsJrjmTqtg9LBFWJcYKt//svfo4plL4UNoWoFTbULct5GedPmYfBJvHJ6PnY4nV7UgvjjhMs+4RG6eL3tqOmnxB3VmhL5A494yHx6YfbRt/iAHiw==; _ym_visorc=b; gsscgib-w-hh=zLG0xSgas3Xl7T5MyY8XPDycx1vIdJOEZaSMbSmzBMifW7bvwEPWDgROKVo+S35SxnKvDzrhq9alJhnFwb/1l0aGlQqkGja3f6yrNB1APX4nP7Kke6xFfFHjQ3XvZXoVSqKgHzOLR79VsmljVLYE4yago/USE7n/++waXH46zuqxqeX/y0iXAmh74N1twEb69DWPbGnq837vEbyMbUxXlhWVbmXIXJspZWqM2VCK2AtveQLEbVLptL22lzcNQQ==; gsscgib-w-hh=1VqwfwRSMN0XqLDy5M61UV/blVoA8iLyCrEwZmWq/wnCVDSH6gEZ8BNmD7EriredJaKHDDzdVkQUO2z9phSBY0lrd+iIeevw+h0nYdxRF2cMinY0J8tOjznKW0nvOkaf6D6NOzacWpikeLZV08sq2iXnULA15SKV4KxIZ95GJM4zaXP0aaoybeerYnio6jKYcVH55vhE7BPxu1SKscmiXPY/5gB0+/Q0xJs+7CdJy8e0jToIC1J3XOOWjZYlxw==; fgsscgib-w-hh=RZjje55ccb9683638fe411189d9c9891c4317354; tmr_detect=0^%^7C1708606149267; cookie_policy_agreement=true',
    'Upgrade-Insecure-Requests': '1',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    # Requests doesn't support trailers
    # 'TE': 'trailers',
}

response = requests.get('https://hh.ru/vacancies/analitik', cookies=cookies, headers=headers)

## <a id='toc1_2_'></a>[Выгрузка вакансий](#toc0_)

In [4]:
df_total_cards = pd.DataFrame()

for page in tqdm(range(40)):
    url = f'https://hh.ru/vacancies/analitik?page={page}&disableBrowserCache=true'
    
    response = requests.get(url, cookies=cookies, headers=headers)
    time.sleep(1)
    soup = BeautifulSoup(response.content, 'html.parser')
    
    total_data = []
    for card in soup.find_all(class_ = 'serp-item'):
        card_url = card.find_all(class_='bloko-link', href=True)[0]['href']
        card_title = card.find_all(class_='bloko-link')[0].text
        try:
            card_salary = card.find_all(class_='bloko-header-section-2')[0].text
        except: 
            card_salary = None
        card_company = card.find_all(class_='vacancy-serp-item-company')[0].text
        card_exp = card.find_all(class_='bloko-h-spacing-container')[1].text

        temp_data = [card_url, card_title, card_salary, card_company, card_exp]
        total_data.append(temp_data)
        
    page_df = pd.DataFrame(total_data, columns=['url', 'title','salary','company', 'exp'])
    
    df_total_cards = pd.concat([page_df, df_total_cards])

  0%|          | 0/40 [00:00<?, ?it/s]

In [5]:
df_total_cards = df_total_cards.reset_index(drop=True)
df_total_cards # просмотр таблицы

,url,title,salary,company,exp
0,https://hh.ru/vacancy/90217739?query=%D0%B0%D0...,Системный аналитик,None,"Russ OutdoorМосква, Динамо и еще 3",Опыт от 3 до 6 лет
1,https://hh.ru/vacancy/91080722?query=%D0%B0%D0...,Аналитик по ценообразованию (недвижимость),None,ГК ФСКМосква,Опыт от 1 года до 3 лет
2,https://hh.ru/vacancy/93284651?query=%D0%B0%D0...,Junior product manager,None,"ТайрайМосква, Менделеевская",Без опыта
3,https://hh.ru/vacancy/92221554?query=%D0%B0%D0...,Бизнес-аналитик,None,ООО ОзбэттеризМосква,Опыт от 1 года до 3 лет
4,https://hh.ru/vacancy/93279573?query=%D0%B0%D0...,Аналитик 1С,от 150 000 ₽,"ООО СДЛ-группМосква, Шоссе Энтузиастов",Опыт от 1 года до 3 лет
...,...,...,...,...,...
1999,https://hh.ru/vacancy/92582364?query=%D0%B0%D0...,Аналитик,None,Syndicate GroupМосква,Без опыта
2000,https://hh.ru/vacancy/93510875?query=%D0%B0%D0...,Аналитик (стратегический консалтинг),None,Global Remote ProfessionalsМосква,Без опыта
2001,https://hh.ru/vacancy/93465209?query=%D0%B0%D0...,Товарный аналитик,None,EfendemМосква,Опыт от 1 года до 3 лет
2002,https://hh.ru/vacancy/93353933?query=%D0%B0%D0...,Excel - аналитик,от 50 000 ₽,"ООО GetBloggerМосква, Крестьянская застава и е...",Опыт от 1 года до 3 лет


In [6]:
# проверка корректности данных на примере одной ссылки
df_total_cards.iloc[1955]['url'] 

'https://hh.ru/vacancy/89221852?query=%D0%B0%D0%BD%D0%B0%D0%BB%D0%B8%D1%82%D0%B8%D0%BA&hhtmFrom=vacancy_search_list'

## <a id='toc1_3_'></a>[Выгрузка доп. информации по выборке вакансий](#toc0_)

In [7]:
total_data_vacancy = []
for vacancy_url in tqdm(df_total_cards.url.unique()[:200]):
    response = requests.get(vacancy_url, cookies=cookies, headers=headers)

    soup = BeautifulSoup(response.content)
    try:
        vacancy = soup.find_all(class_ = 'vacancy-description')[0]
        vacancy_format = soup.find_all(class_='vacancy-description-list-item')[1].text
        try:
            vacancy_address = vacancy.find_all(class_='vacancy-section')[3].text
        except:
            vacancy_address = None
        try:
            vacancy_desc = vacancy.find_all(class_='vacancy-section')[0].text
        except:
            vacancy_desc = None
        try:
            vacancy_keys = vacancy.find_all(class_='vacancy-section')[2].text
        except:
            vacancy_keys = None

        temp_data_vacancy = [vacancy_url, vacancy_format, vacancy_address, vacancy_desc, vacancy_keys]
        total_data_vacancy.append(temp_data_vacancy)
    except:
        temp_data_vacancy = [vacancy_url, None, None, None, None]
        total_data_vacancy.append(temp_data_vacancy)

  0%|          | 0/200 [00:00<?, ?it/s]

In [8]:
df_vacancy = pd.DataFrame(total_data_vacancy, columns=['url', 'format', 'address', 'desc', 'keys'])
df_vacancy # просмотр таблицы

,url,format,address,desc,keys
0,https://hh.ru/vacancy/90217739?query=%D0%B0%D0...,"Полная занятость, гибкий график",None,"Russ Outdoor — крупнейшая в России компания, к...","АдресМосква, Динамо, Петровский парк, Савёловс..."
1,https://hh.ru/vacancy/91080722?query=%D0%B0%D0...,"Полная занятость, полный день",None,В ФСК Регион Департамента продаж и клиентского...,Ключевые навыкиКвартирографияMS OfficeАналитич...
2,https://hh.ru/vacancy/93284651?query=%D0%B0%D0...,"Полная занятость, полный день",None,Франчайзинговая сеть Тайрай - это более 600 та...,"АдресМосква, Менделеевская, Новослободская ули..."
3,https://hh.ru/vacancy/92221554?query=%D0%B0%D0...,"Полная занятость, полный день","АдресМосква, Михалковская улица, 63Бс4Показать...",ООО «ОЗБЭТТЕРИЗ» амбициозная команда специалис...,Ключевые навыкиМоделирование бизнес процессовО...
4,https://hh.ru/vacancy/93279573?query=%D0%B0%D0...,"Полная занятость, полный день",Контактная информацияПоказать контакты,- Требуемый опыт работы: 1–3 лет - Полная заня...,Ключевые навыки1С: Предприятие 8Постановка зад...
...,...,...,...,...,...
195,https://hh.ru/vacancy/91553542?query=%D0%B0%D0...,"Полная занятость, полный день","АдресМосква, Новые Черемушки, Профсоюзная улиц...",\n\n\n\n\n\n\n\nлидер рынка недвижимости Росси...,Контактная информацияПоказать контакты
196,https://hh.ru/vacancy/93353658?query=%D0%B0%D0...,"Полная занятость, полный день","АдресМосква, Воронцовская улица, 35Показать на...",Ищем в свою команду Финансового Аналитика Комп...,Ключевые навыкиФинансовый анализУправленческая...
197,https://hh.ru/vacancy/93670423?query=%D0%B0%D0...,"Полная занятость, полный день",None,Клиника Хадасса ищет в свою команду аналитика ...,"АдресМосква, Сколково, Западный административн..."
198,https://hh.ru/vacancy/92624039?query=%D0%B0%D0...,"Полная занятость, полный день",None,"Вакансия компании: ООО ""НЬЮМЕДИА""Разыскиваем я...",Контактная информацияПоказать контакты


## <a id='toc1_4_'></a>[Информация по формату, опыту и зарплате](#toc0_)

In [9]:
exper = df_total_cards.exp.value_counts().to_frame()
display(exper)

,count
exp,
Опыт от 1 года до 3 лет,1242
Опыт от 3 до 6 лет,545
Без опыта,187
Опыт более 6 лет,30


In [10]:
salary_vc = df_total_cards.salary.value_counts().to_frame()
display(salary_vc)

,count
salary,
от 100 000 ₽,50
от 150 000 ₽,48
от 120 000 ₽,27
от 200 000 ₽,23
от 80 000 ₽,20
...,...
200 000 – 280 000 ₽,1
от 55 000 ₽,1
35 000 – 55 000 ₽,1


In [11]:
form_vc = df_vacancy.format.value_counts().to_frame()
display(form_vc)

,count
format,
"Полная занятость, полный день",133
"Полная занятость, удаленная работа",47
"Полная занятость, гибкий график",11
"Стажировка, полный день",3
"Частичная занятость, удаленная работа",2
"Проектная работа/разовое задание, удаленная работа",1


## <a id='toc1_5_'></a>[Анализ ключевых навыков](#toc0_)

In [12]:
df_vacancy_kyes = df_vacancy.copy()

In [13]:
df_vacancy_kyes['keys'] = df_vacancy_kyes['keys'].astype(str)

In [14]:
df_vacancy_kyes_list = df_vacancy_kyes.loc[(df_vacancy_kyes['keys'].str.contains('Ключевые навыки')),'keys'] \
    .apply(lambda x: x.replace('Ключевые навыки', ''))
df_vacancy_kyes_list.head(20) # корректный список ключевых навыхов (первые 20 пунктов)

1     КвартирографияMS OfficeАналитическое мышлениеА...
3     Моделирование бизнес процессовОптимизация бизн...
4     1С: Предприятие 8Постановка задач разработчика...
5     CorelDRAWMS PowerPointMS ExcelФинансовое модел...
6     Бизнес-процессыBPMNSQLAPIРазработка технически...
9     Аналитические исследованияМоделирование макроэ...
10    Аналитическое мышлениеСбор и анализ информации...
11                          MS SQLMS ExcelАнализ данных
12    MS ExcelУправленческая отчетностьПлан-фактный ...
15                                        Бизнес-анализ
16    ЦенообразованиеАнализ ассортиментной линейкиАн...
17    Статистический анализСоциологические исследова...
18                           UMLBPMNEPCСистемный анализ
22    Финансовый анализАналитическое мышлениеАнализ ...
23    ВнимательностьРабота с базами данныхАнглийский...
24    ARISУправление проектамиMS VisioBPMN1САвтомати...
26                            SQLPythonTableauA/B тесты
30    Управление командойОбучение и развитиеДело

In [15]:
sql_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('SQL|sql')]
print(f'SQL в ключевых навыках встречается в {sql_key.shape[0]} случаях')
python_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('Python|python')]
print(f'Python в ключевых навыках встречается в {python_key.shape[0]} случаях')
excel_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('Excel|excel')]
print(f'Excel в ключевых навыках встречается в {excel_key.shape[0]} случаях')
bi_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('BI|bi|Bi')]
print(f'BI в ключевых навыках встречается в {bi_key.shape[0]} случаях')
eng_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('Английский')]
print(f'Английский язык в ключевых навыках встречается в {eng_key.shape[0]} случаях')
bpmn_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('BPMN|bpmn|Bpmn')]
print(f'BPMN в ключевых навыках встречается в {bpmn_key.shape[0]} случаях')
api_key = df_vacancy_kyes_list.loc[df_vacancy_kyes_list.str.contains('API|api|Api')]
print(f'API в ключевых навыках встречается в {api_key.shape[0]} случаях')

SQL в ключевых навыках встречается в 49 случаях
Python в ключевых навыках встречается в 12 случаях
Excel в ключевых навыках встречается в 27 случаях
BI в ключевых навыках встречается в 11 случаях
Английский язык в ключевых навыках встречается в 20 случаях
BPMN в ключевых навыках встречается в 30 случаях
API в ключевых навыках встречается в 9 случаях


## <a id='toc1_6_'></a>[Анализ описания вакансий](#toc0_)

In [16]:
df = df_total_cards.merge(df_vacancy_kyes, how='right')
df = df.fillna('')
df

,url,title,salary,company,exp,format,address,desc,keys
0,https://hh.ru/vacancy/90217739?query=%D0%B0%D0...,Системный аналитик,,"Russ OutdoorМосква, Динамо и еще 3",Опыт от 3 до 6 лет,"Полная занятость, гибкий график",,"Russ Outdoor — крупнейшая в России компания, к...","АдресМосква, Динамо, Петровский парк, Савёловс..."
1,https://hh.ru/vacancy/91080722?query=%D0%B0%D0...,Аналитик по ценообразованию (недвижимость),,ГК ФСКМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",,В ФСК Регион Департамента продаж и клиентского...,Ключевые навыкиКвартирографияMS OfficeАналитич...
2,https://hh.ru/vacancy/93284651?query=%D0%B0%D0...,Junior product manager,,"ТайрайМосква, Менделеевская",Без опыта,"Полная занятость, полный день",,Франчайзинговая сеть Тайрай - это более 600 та...,"АдресМосква, Менделеевская, Новослободская ули..."
3,https://hh.ru/vacancy/92221554?query=%D0%B0%D0...,Бизнес-аналитик,,ООО ОзбэттеризМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день","АдресМосква, Михалковская улица, 63Бс4Показать...",ООО «ОЗБЭТТЕРИЗ» амбициозная команда специалис...,Ключевые навыкиМоделирование бизнес процессовО...
4,https://hh.ru/vacancy/93279573?query=%D0%B0%D0...,Аналитик 1С,от 150 000 ₽,"ООО СДЛ-группМосква, Шоссе Энтузиастов",Опыт от 1 года до 3 лет,"Полная занятость, полный день",Контактная информацияПоказать контакты,- Требуемый опыт работы: 1–3 лет - Полная заня...,Ключевые навыки1С: Предприятие 8Постановка зад...
...,...,...,...,...,...,...,...,...,...
195,https://hh.ru/vacancy/91553542?query=%D0%B0%D0...,Ведущий маркетолог-аналитик по работе с клиент...,120 000 – 150 000 ₽,"Этажи МоскваМосква, Новые Черемушки",Опыт от 1 года до 3 лет,"Полная занятость, полный день","АдресМосква, Новые Черемушки, Профсоюзная улиц...",\n\n\n\n\n\n\n\nлидер рынка недвижимости Росси...,Контактная информацияПоказать контакты
196,https://hh.ru/vacancy/93353658?query=%D0%B0%D0...,Финансовый аналитик,до 150 000 ₽,ООО ФьютехМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день","АдресМосква, Воронцовская улица, 35Показать на...",Ищем в свою команду Финансового Аналитика Комп...,Ключевые навыкиФинансовый анализУправленческая...
197,https://hh.ru/vacancy/93670423?query=%D0%B0%D0...,Аналитик (премиум клиника Хадасса),,"МедсканМосква, Сколково",Опыт от 1 года до 3 лет,"Полная занятость, полный день",,Клиника Хадасса ищет в свою команду аналитика ...,"АдресМосква, Сколково, Западный административн..."
198,https://hh.ru/vacancy/92624039?query=%D0%B0%D0...,Трейдер-аналитик в хедж-фонд,,JobersМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",,"Вакансия компании: ООО ""НЬЮМЕДИА""Разыскиваем я...",Контактная информацияПоказать контакты


In [17]:
# обработка описания вакансий
df['desc'] = df.desc.str.lower()
df['desc'] = df.desc.apply(lambda x: re.sub(r'[,"_(.)]', '', x))
df['desc'] = df.desc.apply(lambda x: re.sub(r'[\n]', ' ', x))
df['desc'] = df.desc.apply(lambda x: re.sub(r'[\t]', ' ', x))
df['desc'] = df.desc.apply(lambda x: re.sub(r'[/]', ' ', x))
df['desc'] = df.desc.str.replace('  ', ' ')

In [18]:
list_desc = list(df.desc) # список описаний

In [19]:
dic_ = {}
for item in list_desc: # создание словаря слов из описаний
    try:
        item_split = item.split(' ')
        for el in item_split:
            el = str(el)
            if el in dic_:
                dic_[el] += 1
            else:
                dic_[el] = 1
    except:
        continue

In [20]:
df_keys = pd.DataFrame([dic_], index=[0]).T.reset_index() # создание таблицы слов из описаний
df_keys.columns = ['keyword', 'keyword_counts']
df_keys = df_keys.sort_values('keyword_counts', ascending=False)
df_keys['keys_len'] = df_keys.keyword.apply(lambda x: len(x))
df_keys.query('keys_len > 1').head(20)

,keyword,keyword_counts,keys_len
67,на,722,2
136,по,605,2
100,опыт,524,4
101,работы,509,6
94,для,444,3
49,мы,399,2
221,от,269,2
449,знание,252,6
195,данных,216,6
75,анализ,181,6


In [21]:
exc = df.loc[df.desc.str.contains('excel')] # excel в описаниях вакансий
display(exc.head())
exc.shape[0]

,url,title,salary,company,exp,format,address,desc,keys
5,https://hh.ru/vacancy/93164977?query=%D0%B0%D0...,Финансовый аналитик в производственную компани...,,ООО Унечский завод тугоплавких металловМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день","АдресМосква, Бауманская улицаПоказать на больш...",ооо унечский завод тугоплавких металлов являе...,Ключевые навыкиCorelDRAWMS PowerPointMS ExcelФ...
10,https://hh.ru/vacancy/93716832?query=%D0%B0%D0...,Менеджер-аналитик Ozon,от 15 000 ₽,ИП Канунова Мария АндреевнаМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",,обязанности: - мониторинг цен и контроль ценоо...,Ключевые навыкиАналитическое мышлениеСбор и ан...
12,https://hh.ru/vacancy/92906556?query=%D0%B0%D0...,Младший финансовый аналитик,,"Онлайн-школа ТетрикаМосква, Курская",Без опыта,"Полная занятость, полный день","АдресМосква, Курская, Нижняя Сыромятническая у...",привет! мы — онлайн-школа тетрика являемся...,Ключевые навыкиMS ExcelУправленческая отчетнос...
16,https://hh.ru/vacancy/92401342?query=%D0%B0%D0...,Категорийный менеджер в отдел маркетплейсов (FBO),90 000 – 110 000 ₽,ООО Элит ГруппМосква,Опыт от 1 года до 3 лет,"Полная занятость, удаленная работа",Контактная информацияПоказать контакты,компания сантехникатут является одним из крупн...,Ключевые навыкиЦенообразованиеАнализ ассортиме...
17,https://hh.ru/vacancy/93194787?query=%D0%B0%D0...,"Аналитик (Интерн) в практику ""Социология бизнеса""",,"ЭКОПСИ КонсалтингМосква, Дмитровская",Без опыта,"Полная занятость, гибкий график","АдресМосква, Дмитровская, Новодмитровская улиц...",социология бизнеса – одна из практик «экопси к...,Ключевые навыкиСтатистический анализСоциологич...


71

In [22]:
sql = df.loc[df.desc.str.contains('sql')] # sql в описаниях вакансий
display(sql.head())
sql.shape[0]

,url,title,salary,company,exp,format,address,desc,keys
0,https://hh.ru/vacancy/90217739?query=%D0%B0%D0...,Системный аналитик,,"Russ OutdoorМосква, Динамо и еще 3",Опыт от 3 до 6 лет,"Полная занятость, гибкий график",,russ outdoor — крупнейшая в россии компания ко...,"АдресМосква, Динамо, Петровский парк, Савёловс..."
6,https://hh.ru/vacancy/90055162?query=%D0%B0%D0...,Системный аналитик,,"АО Эксперт РАМосква, Таганская",Опыт от 1 года до 3 лет,"Полная занятость, полный день","АдресМосква, Таганская, Николоямская улица, 13...",«эксперт ра» – крупнейшее рейтинговое агентств...,Ключевые навыкиБизнес-процессыBPMNSQLAPIРазраб...
7,https://hh.ru/vacancy/91331861?query=%D0%B0%D0...,Тест-аналитик,200 000 – 300 000 ₽,Diamond Personnel R&CМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день",,для крупного банка подбираем - тест-аналитика ...,None
11,https://hh.ru/vacancy/93208814?query=%D0%B0%D0...,Бизнес-аналитик (debt collection),от 120 000 ₽,ООО МКК КНОПКАДЕНЬГИМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день","АдресМосква, Алтуфьевское шоссе, 27Показать на...",обязанности: развитие внутреннего сервиса взыс...,Ключевые навыкиMS SQLMS ExcelАнализ данных
21,https://hh.ru/vacancy/93676163?query=%D0%B0%D0...,Аналитик в Команду Технорисков,,Сбер для экспертовМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",,команда технорисков сбера развива...,None


65

In [23]:
py = df.loc[df.desc.str.contains('python')] # python в описаниях вакансий
display(py.head())
py.shape[0]

,url,title,salary,company,exp,format,address,desc,keys
21,https://hh.ru/vacancy/93676163?query=%D0%B0%D0...,Аналитик в Команду Технорисков,,Сбер для экспертовМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",,команда технорисков сбера развива...,None
26,https://hh.ru/vacancy/93680741?query=%D0%B0%D0...,Data аналитик,,EmploycityМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день",,международная компания которая разрабатывает н...,Ключевые навыкиSQLPythonTableauA/B тесты
30,https://hh.ru/vacancy/93671331?query=%D0%B0%D0...,Руководитель отдела аналитики,,ООО МедиапоинтМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день","АдресМосква, Замоскворецкая линия, метро Техно...",в международную digital-платформу игровой тема...,Ключевые навыкиУправление командойОбучение и р...
31,https://hh.ru/vacancy/92580946?query=%D0%B0%D0...,Data Analyst,,ARKМосква,Опыт от 3 до 6 лет,"Полная занятость, полный день",,открыта вакансия data analyst у наших партнеро...,Ключевые навыкиSQLPySparkMS SQLPythonPower BIE...
32,https://hh.ru/vacancy/92325010?query=%D0%B0%D0...,Data analyst,до 1 500 $,CLS GlobalМосква,Опыт от 1 года до 3 лет,"Полная занятость, полный день",АдресМоскваПоказать на большой карте,привет! мы - cls global финтех-компания специа...,Ключевые навыкиPythonРабота с большим объемом ...


20